In [1]:
import pdb
import imp
import os
from pathlib import Path
import torch
import numpy as np
from matplotlib import pyplot
import rasterio
import fastai
from multispectral import corine
from multispectral import coords
from multispectral import windows
from multispectral import tools
from multispectral import bands
import neptune
import infra
import zoo

neptune.init('denised/landcover')
corine.set_corine_directory(os.environ['CORINE_DIR'])
windows_list = os.environ['LANDSAT_DIR'] + '/all_windows.csv'

In [2]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [15.0,10.0]

In [3]:
allwindows = windows.from_file(windows_list)
(tr_list, val_list) = windows.randomized_split(allwindows)
print(len(tr_list), len(val_list))

In [6]:
#learner = zoo.Simple.create(tr_list,val_list,[6,20,30,11])
#learner = zoo.ImageUResNet.create(tr_list,val_list)
#learner = infra.LearnerPlus.create_from_file('resnet18.pkl', tr_list, val_list)
learner = zoo.MultiUResNet.create(tr_list,val_list)

In [ ]:
learner.lr_find()

In [8]:
learner.recorder.plot()

In [16]:
?learner.fit_one_cycle

In [7]:
learner.fit_one_cycle(1)

In [21]:
exp = neptune.create_experiment('resnet18 repeat')

In [8]:
learner.export('models/multiresnet18.pkl')

In [9]:
predset = tools.get_prediction_set(learner, tr_list[:20])

In [10]:
collapsedpreds = tools.collapse_predictions(predset)

In [11]:
tools.show_predictions(collapsedpreds)

In [13]:
tools.plot_image_stats(predset.targets,band_labels=bands.CORINE_BANDS)

In [14]:
tools.plot_image_stats(predset.predictions,band_labels=bands.CORINE_BANDS)